In [ ]:
!pip install deepface

DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.11/site-packages/dlib-19.24.99-py3.11-macosx-11.1-arm64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
  Using cached deepface-0.0.93-py3-none-any.whl.metadata (30 kB)
  Using cached gdown-5.2.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached opencv_python-4.11.0.86-cp37-abi3-macosx_13_0_arm64.whl.metadata (20 kB)
  Using cached flask_cors-5.0.1-py3-none-any.whl.metadata (961 bytes)
  Using cached mtcnn-1.0.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached retina_face-0.0.17-py3-none-any.whl.metadata (10 kB)
  Using cached fire-0.7.0.tar.gz (87 kB)
  Preparing metadata (setup.py) ... done
  Using cached gunicorn-23.0.0-py3-none-any.whl.metadata (4.4 kB)
  Using cached joblib-1.5.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached lz4-4.4.4-cp311-cp311-macosx_11_0_arm64.whl.metadata

In [1]:
from deepface import DeepFace

In [2]:
img_url = 'https://public.onlyfans.com/files/j/jg/jgk/jgkdmdkg0szjkhcculq6ru2ijhzfi52i1700978202/35325663/avatar.jpg'
male_image_url='https://public.onlyfans.com/files/q/q6/q6l/q6lop81emnfpa6pvwp7oleay75vtjjhu1743222977/41828953/avatar.jpg'
female_image_url='https://public.onlyfans.com/files/e/ee/eex/eexjdcl1yxscnbwutvtz1lidvbixrycc1703421470/266970621/avatar.jpg'

In [3]:
demography = DeepFace.analyze(male_image_url,actions=['gender'])

In [4]:
demography

[{'gender': {'Woman': 0.14164545573294163, 'Man': 99.85835552215576},
  'dominant_gender': 'Man',
  'region': {'x': 690,
   'y': 1363,
   'w': 1015,
   'h': 1015,
   'left_eye': (1380, 1780),
   'right_eye': (1008, 1717)},
  'face_confidence': 0.95}]

In [10]:
def analyze_gender_from_image(image_url):
    try:
        print(f"Running DeepFace for img_url: {image_url}")
        result = DeepFace.analyze(image_url, actions=['gender'])
        print("Result:", result)
        gender = result[0]['gender']
        confidence = result[0]['gender_confidence'] / 100
        return {'gender': gender.lower(), 'confidence': round(confidence, 2)}

    except Exception as e:
        print("Error:", e)
        return {'gender': 'unknown', 'confidence': 0.0, 'error': str(e)}

In [12]:
analyze_gender_from_image(img_url)

Running DeepFace for img_url: https://public.onlyfans.com/files/x/xi/xiz/xiz0ygyxebfncys6zvlybgnzx4xzhlb71609974360/avatar.jpg
Result: [{'gender': {'Woman': 87.94256448745728, 'Man': 12.057441473007202}, 'dominant_gender': 'Woman', 'region': {'x': 313, 'y': 884, 'w': 528, 'h': 528, 'left_eye': (648, 1100), 'right_eye': (475, 1102)}, 'face_confidence': 0.89}]
Error: 'gender_confidence'


{'gender': 'unknown', 'confidence': 0.0, 'error': "'gender_confidence'"}

In [1]:
!pip install mediapipe

DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.11/site-packages/dlib-19.24.99-py3.11-macosx-11.1-arm64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
  Using cached sounddevice-0.5.1-py3-none-macosx_10_6_x86_64.macosx_10_6_universal2.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 MB 3.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.2/394.2 kB 3.9 MB/s eta 0:00:00a 0:00:01
Using cached sounddevice-0.5.1-py3-none-macosx_10_6_x86_64.macosx_10_6_universal2.whl (107 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 MB 3.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 3.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninsta

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import requests
from io import BytesIO

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True)

def infer_gender_from_body_url(image_url):
    try:
        # Download image
        response = requests.get(image_url)
        if response.status_code != 200:
            return "Failed to download image."

        image_array = np.asarray(bytearray(response.content), dtype=np.uint8)
        image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

        if image is None:
            return "Failed to decode image."

        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        result = pose.process(image_rgb)

        if not result.pose_landmarks:
            return "No body detected."

        landmarks = result.pose_landmarks.landmark

        def get_coord(lm):
            return np.array([lm.x, lm.y])

        # Get shoulder and hip points
        left_shoulder = get_coord(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER])
        right_shoulder = get_coord(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER])
        left_hip = get_coord(landmarks[mp_pose.PoseLandmark.LEFT_HIP])
        right_hip = get_coord(landmarks[mp_pose.PoseLandmark.RIGHT_HIP])

        shoulder_width = np.linalg.norm(left_shoulder - right_shoulder)
        hip_width = np.linalg.norm(left_hip - right_hip)
        ratio = shoulder_width / hip_width if hip_width > 0 else 0

        # Basic heuristic
        if ratio > 1.15:
            return "Likely Male"
        elif ratio < 1.0:
            return "Likely Female"
        else:
            return "Uncertain Gender"

    except Exception as e:
        return f"Error: {str(e)}"

# Example usage:
# print(infer_gender_from_body_url("https://example.com/image.jpg"))

I0000 00:00:1746891204.062030 44542884 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1


W0000 00:00:1746891204.164084 44560810 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746891204.178121 44560810 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [15]:
infer_gender_from_body_url(img_url)

'Likely Male'

In [21]:
def infer_gender_deepface(image_url):
    try:
        response = requests.get(image_url)
        if response.status_code != 200:
            return "Failed to download image."


        # DeepFace can analyze gender with enforce_detection=False
        result = DeepFace.analyze(img_path=image_url, actions=['gender'], enforce_detection=False)

        gender = result[0]['dominant_gender']
        return f"Predicted Gender: {gender.capitalize()}"

    except Exception as e:
        return f"Error: {str(e)}"


In [29]:
infer_gender_deepface('https://public.onlyfans.com/files/c/ca/cak/cakrab2djlhvm1m5dpv5pncxjysyxw111683091395/315516647/avatar.jpg')

'Predicted Gender: Man'

In [24]:
!pip install transformers

DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.11/site-packages/dlib-19.24.99-py3.11-macosx-11.1-arm64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 4.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.3/484.3 kB 4.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.4/418.4 kB 4.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 4.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 4.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.3
    Uninstalling huggingface-hub-0.24.3:
      Successfully uninstalled huggingface-hub-0.24.3


In [28]:
import torch
from transformers import AutoFeatureExtractor, AutoModelForImageClassification
from PIL import Image
import requests
from io import BytesIO
import json
import torch.nn.functional as F

# Load Hugging Face DINO-ViTB16 Model (Pretrained + Fine-tuned for Image Classification)
class GenderClassifier:
    def __init__(self):
        self.extractor = AutoFeatureExtractor.from_pretrained("facebook/dino-vitb16")
        self.model = AutoModelForImageClassification.from_pretrained("facebook/dino-vitb16")
        self.model.eval()

    def predict_gender(self, image_url: str) -> dict:
        response = requests.get(image_url)
        img = Image.open(BytesIO(response.content)).convert("RGB")
        
        # Preprocess the image
        inputs = self.extractor(img, return_tensors="pt")
        
        # Get model output
        with torch.no_grad():
            outputs = self.model(**inputs)
        
        logits = outputs.logits
        probabilities = F.softmax(logits, dim=-1)  # Convert logits to probabilities
        
        # Get the predicted class (gender) and its confidence
        predicted_class = torch.argmax(probabilities, dim=-1).item()
        confidence = probabilities[0][predicted_class].item()

        # Map the predicted class to gender (assuming 0=male, 1=female, etc.)
        gender = "female" if predicted_class == 1 else "male"  # Adjust based on your actual labels
        
        return {
            "gender": gender,
            "confidence": confidence
        }

# Wrapper for your input format
def classify_gender_from_input(user_data: dict):
    avatar_url = user_data["ProfileDatum"]["avatar"]
    classifier = GenderClassifier()
    return classifier.predict_gender(avatar_url)

# Example input
input_json = '''
{
    "username": "aaroncashxxx",
    "ProfileDatum": {
        "id": "77517",
        "full_name": "Aaron Cash",
        "avatar": "https://public.onlyfans.com/files/c/ca/cak/cakrab2djlhvm1m5dpv5pncxjysyxw111683091395/315516647/avatar.jpg",
        "banner": "https://thumbs.onlyfans.com/public/files/thumbs/w760/j/j4/j45/j45ddjmugwcupixazjyst4pbnx6jeav41683091029/315516647/header.jpg",
        "about": "Your favorite Vers Cuban living in Miami..."
    }
}
'''

user = json.loads(input_json)
result = classify_gender_from_input(user)
print(f"Predicted gender: {result['gender']} with confidence: {result['confidence']}")


Some weights of ViTForImageClassification were not initialized from the model checkpoint at facebook/dino-vitb16 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Predicted gender: female with confidence: 0.6721607446670532


In [43]:
!pip install insightface onnxruntime typing_extensions

DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.11/site-packages/dlib-19.24.99-py3.11-macosx-11.1-arm64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [45]:
pip install --upgrade typing_extensions


DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.11/site-packages/dlib-19.24.99-py3.11-macosx-11.1-arm64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
Note: you may need to restart the kernel to use updated packages.


In [9]:
import insightface
import requests
from PIL import Image
from io import BytesIO
import numpy as np

face_analyzer = insightface.app.FaceAnalysis(name='buffalo_l', providers=['CPUExecutionProvider'])
face_analyzer.prepare(ctx_id=0)
def analyze_gender_from_image(image_url):
    try:
        response = requests.get(image_url)
        response.raise_for_status()

        image = Image.open(BytesIO(response.content)).convert("RGB")
        image_np = np.array(image)

        faces = face_analyzer.get(image_np)

        if not faces:
            return {'gender': 'unknown', 'confidence': 0.0, 'error': 'No face detected'}

        # Assume first face (you can modify this if needed)
        face = faces[0]
        print('face', face)
        gender_label = 'male' if face.sex == 1 else 'female'
        return {'gender': gender_label, 'confidence': float(face.det_score)}

    except Exception as e:
        print('image_url',image_url)
        return {'gender': 'unknown', 'confidence': 0.0, 'error': str(e)}

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/chestergarettcalingacion/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/chestergarettcalingacion/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/chestergarettcalingacion/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/chestergarettcalingacion/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/chestergarettcalingacio

In [13]:
male_image_url='https://public.onlyfans.com/files/q/q6/q6l/q6lop81emnfpa6pvwp7oleay75vtjjhu1743222977/41828953/avatar.jpg'
female_image_url='https://public.onlyfans.com/files/e/ee/eex/eexjdcl1yxscnbwutvtz1lidvbixrycc1703421470/266970621/avatar.jpg'
analyze_gender_from_image(female_image_url)

face {'bbox': array([391.22385,  51.88667, 652.1092 , 434.11658], dtype=float32), 'kps': array([[471.23022, 221.15161],
       [592.4004 , 219.78415],
       [539.2211 , 308.90967],
       [497.17145, 357.4982 ],
       [578.5274 , 354.23663]], dtype=float32), 'det_score': 0.8491061, 'landmark_3d_68': array([[ 3.77332642e+02,  1.89552017e+02,  1.89827713e+02],
       [ 3.82980255e+02,  2.31540283e+02,  1.91577499e+02],
       [ 3.89350433e+02,  2.74366852e+02,  1.88920090e+02],
       [ 3.96946228e+02,  3.15689331e+02,  1.80027298e+02],
       [ 4.09920654e+02,  3.55832367e+02,  1.54799957e+02],
       [ 4.30073273e+02,  3.87307159e+02,  1.31088318e+02],
       [ 4.55790558e+02,  4.10060608e+02,  1.08996025e+02],
       [ 4.87774567e+02,  4.27312988e+02,  7.90753021e+01],
       [ 5.31488525e+02,  4.39616302e+02,  6.80467453e+01],
       [ 5.75752563e+02,  4.31961914e+02,  8.18213654e+01],
       [ 6.13203430e+02,  4.18045685e+02,  1.17107285e+02],
       [ 6.40643188e+02,  3.93616760e

{'gender': 'female', 'confidence': 0.8491060733795166}